In [19]:
import pandas as pd
import os

In [4]:
data=pd.read_csv('Desktop/Business Spark/data/raw/train.csv')
data.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [5]:
unwanted_cols=['Raw ID','Order ID','Customer ID','Customer Name','Product ID','Product Name','Postal Code','Country']

In [6]:
cldata=data.drop(columns=[c for c in unwanted_cols if c in data.columns])

In [7]:
cldata.head()

,Row ID,Order Date,Ship Date,Ship Mode,Segment,City,State,Region,Category,Sub-Category,Sales
0,1,08/11/2017,11/11/2017,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Bookcases,261.9600
1,2,08/11/2017,11/11/2017,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Chairs,731.9400
2,3,12/06/2017,16/06/2017,Second Class,Corporate,Los Angeles,California,West,Office Supplies,Labels,14.6200
3,4,11/10/2016,18/10/2016,Standard Class,Consumer,Fort Lauderdale,Florida,South,Furniture,Tables,957.5775
4,5,11/10/2016,18/10/2016,Standard Class,Consumer,Fort Lauderdale,Florida,South,Office Supplies,Storage,22.3680


In [8]:
cldata['Order Date']=pd.to_datetime(cldata['Order Date'],dayfirst=True)
cldata['Ship Date']=pd.to_datetime(cldata['Ship Date'],dayfirst=True)

In [9]:
cldata['year']=cldata['Order Date'].dt.year
cldata['month']=cldata['Order Date'].dt.month
cldata['quarter']=cldata['Order Date'].dt.quarter
cldata['day_of_week']=cldata['Order Date'].dt.day_of_week

In [10]:
cldata.head()

,Row ID,Order Date,Ship Date,Ship Mode,Segment,City,State,Region,Category,Sub-Category,Sales,year,month,quarter,day_of_week
0,1,2017-11-08,2017-11-11,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Bookcases,261.9600,2017,11,4,2
1,2,2017-11-08,2017-11-11,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Chairs,731.9400,2017,11,4,2
2,3,2017-06-12,2017-06-16,Second Class,Corporate,Los Angeles,California,West,Office Supplies,Labels,14.6200,2017,6,2,0
3,4,2016-10-11,2016-10-18,Standard Class,Consumer,Fort Lauderdale,Florida,South,Furniture,Tables,957.5775,2016,10,4,1
4,5,2016-10-11,2016-10-18,Standard Class,Consumer,Fort Lauderdale,Florida,South,Office Supplies,Storage,22.3680,2016,10,4,1


In [11]:
cldata['delivery_days']=(cldata['Ship Date']-cldata['Order Date']).dt.days

In [12]:
cldata.head()

,Row ID,Order Date,Ship Date,Ship Mode,Segment,City,State,Region,Category,Sub-Category,Sales,year,month,quarter,day_of_week,delivery_days
0,1,2017-11-08,2017-11-11,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Bookcases,261.9600,2017,11,4,2,3
1,2,2017-11-08,2017-11-11,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Chairs,731.9400,2017,11,4,2,3
2,3,2017-06-12,2017-06-16,Second Class,Corporate,Los Angeles,California,West,Office Supplies,Labels,14.6200,2017,6,2,0,4
3,4,2016-10-11,2016-10-18,Standard Class,Consumer,Fort Lauderdale,Florida,South,Furniture,Tables,957.5775,2016,10,4,1,7
4,5,2016-10-11,2016-10-18,Standard Class,Consumer,Fort Lauderdale,Florida,South,Office Supplies,Storage,22.3680,2016,10,4,1,7


In [15]:
monthly=(cldata.set_index('Order Date').resample('ME').agg({
    'Sales':'sum',
    'month':'first',
    'quarter':'first',
    'year':'first'
}).reset_index()
)

In [16]:
monthly['lag_1']=montly['Sales'].shift(1)
monthly['lag_2']=montly['Sales'].shift(2)
monthly['lag_3']=montly['Sales'].shift(3)

In [17]:
monthly['rolling_mean_3']=montly['Sales'].rolling(3).mean()
monthly['rolling_std_3']=montly['Sales'].rolling(3).std()

In [18]:
montly=monthly.dropna().reset_index(drop=False)

In [21]:
os.makedirs('Desktop/Business Spark/data/processed',exist_ok=True)
monthly.to_csv('Desktop/Business Spark/data/processed/monthly.csv',index=False)

In [22]:
os.makedirs('Desktop/Business Spark/data/cleaned',exist_ok=True)
cldata.to_csv('Desktop/Business Spark/data/cleaned/clean.csv',index=False)

In [24]:
sample=cldata.sample(100,random_state=42)
os.makedirs('Desktop/Business Spark/data/sample',exist_ok=True)
sample.to_csv('Desktop/Business Spark/data/sample/clean_sample.csv',index=False)